# Imports

In [1]:
import pandas as pd
import numpy as np
import json
import requests

**installing spotipy**

In [2]:
!pip install spotipy

In [3]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

# Setting the credencials

In [4]:
client_id = '2753defd10e2496f80b0d2348c2b5481'
client_secret = '34a98fb5da1b4c7698f93c077db6b127'

sp = spotipy.Spotify(client_credentials_manager=SpotifyClientCredentials(client_id=client_id, client_secret=client_secret))


# Prepare data

In [43]:
# Playlist id
pl_id = 'spotify:playlist:37i9dQZF1DX18jTM2l2fJY'

# Getting the tracks in the playlist
response = sp.playlist(pl_id)

# How many tracks in the playlist
len(response['tracks']['items'])

50

In [ ]:
 response['tracks']['items'][1]

## Getting features ready to turn into dataframe

In [7]:
name = [response['tracks']['items'][i]['track']['name'] for i in range(len(response['tracks']['items']))]

In [8]:
artist = [response['tracks']['items'][i]['track']['artists'][0]['name'] for i in range(len(response['tracks']['items']))]

In [9]:
artist_id = [response['tracks']['items'][i]['track']['album']['artists'][0]['id'] for i in range(len(response['tracks']['items']))]

In [10]:
popularity = [response['tracks']['items'][i]['track']['popularity'] for i in range(len(response['tracks']['items']))]

In [11]:
Id = [response['tracks']['items'][i]['track']['id'] for i in range(len(response['tracks']['items']))]

In [12]:
release_date = [response['tracks']['items'][i]['track']['album']['release_date'] for i in range(len(response['tracks']['items']))]

In [13]:
preview_url = [response['tracks']['items'][i]['track']['preview_url'] for i in range(len(response['tracks']['items']))]

In [14]:
cover_album = [response['tracks']['items'][i]['track']['album']['images'][1]['url'] for i in range(len(response['tracks']['items']))]

In [15]:
explicit = [response['tracks']['items'][i]['track']['explicit'] for i in range(len(response['tracks']['items']))]

### 1st Dataframe ready

In [16]:
data = {}

# Setting the features
data['name'] = name
data['popularity'] = popularity
data['artist'] = artist
data['artist_id'] = artist_id
data['explicit'] = explicit
data['id'] = Id
data['release_date'] = release_date
data['preview_url'] = preview_url
data['cover_album'] = cover_album

# turning into dataframe
df = pd.DataFrame(data)

#turninge the realise date into datetime
df['release_date'] = pd.to_datetime(df['release_date'])

# Turning explicit column in 0 or 1
df['explicit'] = df['explicit'].map(lambda x: 1 if x == True else 0)

### 2nd Datarame ready

In [17]:
# List of features set
tracks = []

# For each track in the 1st dataframe 
for track_id in df.loc[:,'id']:
    track = f'spotify:track:{track_id}' # Get the track features
    audio_features = sp.audio_features(track) # Save in audio features variable
    tracks.append(audio_features[0]) # Append to the tracks list

In [18]:
# New dataframe with all the tracks in the first dataframe
df_features =  pd.DataFrame(tracks)
df_features.head()

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature
0,0.520,0.731,6,-5.338,0,0.0557,0.3420,0.001010,0.3110,0.662,173.930,audio_features,4LRPiXqCikLlN15c3yImP7,spotify:track:4LRPiXqCikLlN15c3yImP7,https://api.spotify.com/v1/tracks/4LRPiXqCikLl...,https://api.spotify.com/v1/audio-analysis/4LRP...,167303,4
1,0.761,0.525,11,-6.900,1,0.0944,0.4400,0.000007,0.0921,0.531,80.870,audio_features,02MWAaffLxlfxAUY7c5dvx,spotify:track:02MWAaffLxlfxAUY7c5dvx,https://api.spotify.com/v1/tracks/02MWAaffLxlf...,https://api.spotify.com/v1/audio-analysis/02MW...,238805,4
2,0.591,0.764,1,-5.484,1,0.0483,0.0383,0.000000,0.1030,0.478,169.928,audio_features,5HCyWlXZPP0y6Gqq8TgA20,spotify:track:5HCyWlXZPP0y6Gqq8TgA20,https://api.spotify.com/v1/tracks/5HCyWlXZPP0y...,https://api.spotify.com/v1/audio-analysis/5HCy...,141806,4
3,0.911,0.712,1,-5.105,0,0.0817,0.0901,0.000027,0.0933,0.425,92.005,audio_features,6Sq7ltF9Qa7SNFBsV5Cogx,spotify:track:6Sq7ltF9Qa7SNFBsV5Cogx,https://api.spotify.com/v1/tracks/6Sq7ltF9Qa7S...,https://api.spotify.com/v1/audio-analysis/6Sq7...,178567,4
4,0.650,0.715,5,-5.198,0,0.2530,0.0993,0.000291,0.1260,0.187,106.672,audio_features,1IHWl5LamUGEuP4ozKQSXZ,spotify:track:1IHWl5LamUGEuP4ozKQSXZ,https://api.spotify.com/v1/tracks/1IHWl5LamUGE...,https://api.spotify.com/v1/audio-analysis/1IHW...,243717,4


### Merge and Cleaning

In [19]:
# Inner merging between the 2 dataframe on id
data = df.merge(df_features, on='id')

# Dropping columns 
data.drop(['type', 'uri', 'track_href', 'analysis_url', 'time_signature'], axis=1, inplace=True)

data.head()

,name,popularity,artist,artist_id,explicit,id,release_date,preview_url,cover_album,danceability,...,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms
0,As It Was,90,Harry Styles,6KImCVD70vtIoJWnq6nGn3,0,4LRPiXqCikLlN15c3yImP7,2022-03-31,https://p.scdn.co/mp3-preview/c43dd07043b29e80...,https://i.scdn.co/image/ab67616d00001e02b46f74...,0.520,...,6,-5.338,0,0.0557,0.3420,0.001010,0.3110,0.662,173.930,167303
1,Heat Waves,84,Glass Animals,4yvcSjfu4PC0CYQyLy4wSq,0,02MWAaffLxlfxAUY7c5dvx,2020-08-06,None,https://i.scdn.co/image/ab67616d00001e029e495f...,0.761,...,11,-6.900,1,0.0944,0.4400,0.000007,0.0921,0.531,80.870,238805
2,STAY (with Justin Bieber),85,The Kid LAROI,2tIP7SsRs7vjIcLrU85W8J,1,5HCyWlXZPP0y6Gqq8TgA20,2021-07-09,https://p.scdn.co/mp3-preview/718a2eb032524494...,https://i.scdn.co/image/ab67616d00001e0241e31d...,0.591,...,1,-5.484,1,0.0483,0.0383,0.000000,0.1030,0.478,169.928,141806
3,Me Porto Bonito,90,Bad Bunny,4q3ewBCX7sLwd24euuV69X,1,6Sq7ltF9Qa7SNFBsV5Cogx,2022-05-06,https://p.scdn.co/mp3-preview/5c80aedfe75d2bc1...,https://i.scdn.co/image/ab67616d00001e0249d694...,0.911,...,1,-5.105,0,0.0817,0.0901,0.000027,0.0933,0.425,92.005,178567
4,Tití Me Preguntó,89,Bad Bunny,4q3ewBCX7sLwd24euuV69X,1,1IHWl5LamUGEuP4ozKQSXZ,2022-05-06,https://p.scdn.co/mp3-preview/53a6217761f8fdfc...,https://i.scdn.co/image/ab67616d00001e0249d694...,0.650,...,5,-5.198,0,0.2530,0.0993,0.000291,0.1260,0.187,106.672,243717


In [20]:
# Get the data from kaggle to compare
df_tracks = pd.read_csv('/home/mdbruchard/code/dmassonfr/Spotify_hit_predictor/raw_data/tracks.csv')
df_tracks.head()

,id,name,popularity,duration_ms,explicit,artists,id_artists,release_date,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature
0,35iwgR4jXetI318WEWsa1Q,Carve,6,126903,0,['Uli'],['45tIt06XoI0Iio4LBEVpls'],1922-02-22,0.645,0.4450,0,-13.338,1,0.4510,0.674,0.7440,0.151,0.127,104.851,3
1,021ht4sdgPcrDgSk7JTbKY,Capítulo 2.16 - Banquero Anarquista,0,98200,0,['Fernando Pessoa'],['14jtPCOoNZwquk5wd9DxrY'],1922-06-01,0.695,0.2630,0,-22.136,1,0.9570,0.797,0.0000,0.148,0.655,102.009,1
2,07A5yehtSnoedViJAZkNnc,Vivo para Quererte - Remasterizado,0,181640,0,['Ignacio Corsini'],['5LiOoJbxVSAMkBS2fUm3X2'],1922-03-21,0.434,0.1770,1,-21.180,1,0.0512,0.994,0.0218,0.212,0.457,130.418,5
3,08FmqUhxtyLTn6pAh6bk45,El Prisionero - Remasterizado,0,176907,0,['Ignacio Corsini'],['5LiOoJbxVSAMkBS2fUm3X2'],1922-03-21,0.321,0.0946,7,-27.961,1,0.0504,0.995,0.9180,0.104,0.397,169.980,3
4,08y9GfoqCWfOGsKdwojr5e,Lady of the Evening,0,163080,0,['Dick Haymes'],['3BiJGZsyX9sJchTqcSA7Su'],1922,0.402,0.1580,3,-16.900,0,0.0390,0.989,0.1300,0.311,0.196,103.220,4


In [24]:
df_tracks['id_artists'] = df_tracks['id_artists'].map(lambda x: x.strip("'[]'"))

In [25]:
df_tracks.head(50)

,id,name,popularity,duration_ms,explicit,artists,id_artists,release_date,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature
0,35iwgR4jXetI318WEWsa1Q,Carve,6,126903,0,['Uli'],45tIt06XoI0Iio4LBEVpls,1922-02-22,0.645,0.4450,0,-13.338,1,0.4510,0.674,0.744000,0.1510,0.1270,104.851,3
1,021ht4sdgPcrDgSk7JTbKY,Capítulo 2.16 - Banquero Anarquista,0,98200,0,['Fernando Pessoa'],14jtPCOoNZwquk5wd9DxrY,1922-06-01,0.695,0.2630,0,-22.136,1,0.9570,0.797,0.000000,0.1480,0.6550,102.009,1
2,07A5yehtSnoedViJAZkNnc,Vivo para Quererte - Remasterizado,0,181640,0,['Ignacio Corsini'],5LiOoJbxVSAMkBS2fUm3X2,1922-03-21,0.434,0.1770,1,-21.180,1,0.0512,0.994,0.021800,0.2120,0.4570,130.418,5
3,08FmqUhxtyLTn6pAh6bk45,El Prisionero - Remasterizado,0,176907,0,['Ignacio Corsini'],5LiOoJbxVSAMkBS2fUm3X2,1922-03-21,0.321,0.0946,7,-27.961,1,0.0504,0.995,0.918000,0.1040,0.3970,169.980,3
4,08y9GfoqCWfOGsKdwojr5e,Lady of the Evening,0,163080,0,['Dick Haymes'],3BiJGZsyX9sJchTqcSA7Su,1922,0.402,0.1580,3,-16.900,0,0.0390,0.989,0.130000,0.3110,0.1960,103.220,4
5,0BRXJHRNGQ3W4v9frnSfhu,Ave Maria,0,178933,0,['Dick Haymes'],3BiJGZsyX9sJchTqcSA7Su,1922,0.227,0.2610,5,-12.343,1,0.0382,0.994,0.247000,0.0977,0.0539,118.891,4
6,0Dd9ImXtAtGwsmsAD69KZT,La Butte Rouge,0,134467,0,['Francis Marty'],2nuMRGzeJ5jJEKlfS7rZ0W,1922,0.510,0.3550,4,-12.833,1,0.1240,0.965,0.000000,0.1550,0.7270,85.754,5
7,0IA0Hju8CAgYfV1hwhidBH,La Java,0,161427,0,['Mistinguett'],4AxgXfD7ISvJSTObqm4aIE,1922,0.563,0.1840,4,-13.757,1,0.0512,0.993,0.000016,0.3250,0.6540,133.088,3
8,0IgI1UCz84pYeVetnl1lGP,Old Fashioned Girl,0,310073,0,['Greg Fieler'],5nWlsH5RDgFuRAiDeOFVmf,1922,0.488,0.4750,0,-16.222,0,0.0399,0.620,0.006450,0.1070,0.5440,139.952,4
9,0JV4iqw2lSKJaHBQZ0e5zK,Martín Fierro - Remasterizado,0,181173,0,['Ignacio Corsini'],5LiOoJbxVSAMkBS2fUm3X2,1922-03-29,0.548,0.0391,6,-23.228,1,0.1530,0.996,0.933000,0.1480,0.6120,75.595,3


In [ ]:
track_ids = ['spotify:track:' + t_id for t_id in df_tracks['id']]
batch_size = 100
preview_urls = []
for i in range(0, len(track_ids), batch_size):
    batch_ids = track_ids[i:i+batch_size]
    batch_tracks = sp.tracks(batch_ids)['tracks']
    batch_urls = [track['preview_url'] for track in batch_tracks]
    preview_urls.extend(batch_urls)

df_tracks['track_sample'] = preview_urls

In [65]:
df_tracks[1000:1050]

,id,name,popularity,duration_ms,explicit,artists,id_artists,release_date,danceability,energy,...,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature,track_sample
1000,5hKoJAuCgukMBXd0W9ZdVt,Ca n'est pas la même chose,0,139293,0,['Maurice Chevalier'],3QXjI80udwDSEfxK5BjUXc,1924,0.476,0.1490,...,-16.185,1,0.0492,0.9960,0.000000,0.3880,0.697,86.447,4,https://p.scdn.co/mp3-preview/19ecc13cab9d4344...
1001,5hoLIJF3clip0VplseKOyG,You're My Ideal,0,195667,0,['Lionel Hampton Orchestra'],6UftTZlUzZBBl2ctjzLQ1w,1924,0.760,0.3540,...,-12.788,0,0.0711,0.7320,0.000052,0.0854,0.775,113.981,4,NaN
1002,5icxP1uv1JMz87JGEpIcHy,Chapter 125 - The Unlit Lamp,0,185649,0,"['Radclyffe Hall', 'Alice White', 'Synthesized...","2OJEUmIIPsTAUjil4dOz70', '35AgMyuKAQIoMJZEp3lN...",1924,0.713,0.3010,...,-16.374,1,0.9170,0.7240,0.000000,0.1130,0.308,115.116,5,NaN
1003,5ixjL1GSioVd8lGNkLwYDM,Say Arabella - What's A Fella To Do,0,176107,0,['George Olsen'],45tQpWrlBy4DMpfAriqfcC,1924-01-01,0.778,0.3800,...,-12.082,1,0.0380,0.9820,0.431000,0.0922,0.963,107.463,4,NaN
1004,5jVl7lFp2QAW3U627qzmpe,Chapter 44 - The Unlit Lamp,0,187570,0,"['Radclyffe Hall', 'Alice White', 'Synthesized...","2OJEUmIIPsTAUjil4dOz70', '35AgMyuKAQIoMJZEp3lN...",1924,0.713,0.2300,...,-16.520,1,0.9210,0.6870,0.000000,0.0925,0.513,128.981,5,NaN
1005,5kSeMwoKtS12E5CM2nKva0,Llanto de Amor - Remasterizado,0,171347,0,['Francisco Canaro'],2maQMqxNnlRrBrS1oAsrX9,1924-07-12,0.423,0.4410,...,-12.383,0,0.0447,0.9960,0.914000,0.7880,0.731,72.046,4,NaN
1006,5kgUhsCxWKTIl9HitccHTw,Button Up Your Overcoat,0,208240,0,"['The Chenille Sisters', ""James Dapogny's Chic...","0WdNVS1rOeusXfVUx3tYOL', '7B9i9Gl6Af1wPWjTD1VQMD",1924,0.520,0.3810,...,-11.147,1,0.0506,0.8140,0.022700,0.2010,0.862,184.814,4,NaN
1007,5moL6Ktb31FOxHgpY6PaLz,Chapter 11 - The Unlit Lamp,0,182865,0,"['Radclyffe Hall', 'Alice White', 'Synthesized...","2OJEUmIIPsTAUjil4dOz70', '35AgMyuKAQIoMJZEp3lN...",1924,0.634,0.2910,...,-16.698,0,0.9480,0.7360,0.000000,0.1660,0.335,85.164,3,NaN
1008,5nEG47pZrOxPo2HQLxnOFn,Beale Street Blues,0,186040,0,['George Olsen'],45tQpWrlBy4DMpfAriqfcC,1924-01-01,0.595,0.4070,...,-11.626,1,0.0654,0.9930,0.256000,0.0874,0.921,200.905,4,NaN
1009,5oK7hYOkBEkXz9d9lumSGd,Cascabelito - Remasterizado,0,169333,0,['Francisco Canaro'],2maQMqxNnlRrBrS1oAsrX9,1924-07-11,0.653,0.2110,...,-16.294,0,0.0660,0.9960,0.862000,0.2230,0.549,123.960,4,NaN


In [ ]:
df_tracks.to_csv('/home/mdbruchard/code/dmassonfr/Spotify_hit_predictor/raw_data/tracks_sample.csv', index=False)